In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, LeakyReLU

from keras.preprocessing.image import img_to_array, load_img
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import boto3


Using TensorFlow backend.


In [2]:
bucket = '<Your Bucket here>'
client = boto3.client('s3')

In [3]:
channels = 3
nb_classes = 1
target_height=180
target_width=320

In [4]:
obj = client.get_object(Bucket=bucket, Key='<path to data csv>')


allData = pd.read_csv(obj['Body'])
fields = ['filename', 'human']
data = allData[fields]

data.head()

,filename,human
0,01-20190707053407-06.jpg,0
1,01-20190707104301-05.jpg,1
2,01-20190707142157-05.jpg,1
3,01-20190707160533-02.jpg,1
4,01-20190713144837-02.jpg,1


In [5]:
dataset = np.ndarray(shape=(len(data), target_height, target_width, channels), dtype=np.float32)
y_dataset = []

# Set of markers so I can create an lst
i=0
for index, row in data.iterrows():
    y_dataset.append(row.human)
    object_path = '<path to image directory>/{}'.format(row.filename)
    obj = client.get_object(Bucket=bucket, Key=object_path)
    img = load_img(obj['Body'], target_size=(target_height, target_width))
    x = img_to_array(img)
    x = x / 255.0
    dataset[i] = x
    i += 1


In [6]:
x_train, x_val_test, y_train, y_val_test = train_test_split(dataset, y_dataset, test_size=0.3)

In [7]:
validation_length = int(int(len(x_val_test)) / 2)

# Separate validation from test
x_val = x_val_test[:validation_length]
y_val = y_val_test[:validation_length]

x_test = x_val_test[validation_length:]
y_test = y_val_test[validation_length:]

print(x_train.shape, x_val.shape, x_test.shape)

(797, 180, 320, 3) (171, 180, 320, 3) (171, 180, 320, 3)


In [8]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=x_train[1,:].shape))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

W0905 17:55:53.497073 140690488915776 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0905 17:55:53.511845 140690488915776 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 17:55:53.513712 140690488915776 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0905 17:55:53.524643 140690488915776 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:39

In [9]:
batch_size=16
epochs=10

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(x_val,y_val))
model.save('human_not_human.h5')

Train on 797 samples, validate on 171 samples
Epoch 1/10
797/797 [==============================] - 14s 17ms/step - loss: 1.5408 - acc: 0.6487 - val_loss: 0.6246 - val_acc: 0.6433
Epoch 2/10
797/797 [==============================] - 12s 15ms/step - loss: 0.5773 - acc: 0.7591 - val_loss: 0.7133 - val_acc: 0.5965
Epoch 3/10
797/797 [==============================] - 12s 15ms/step - loss: 0.4047 - acc: 0.8419 - val_loss: 1.6744 - val_acc: 0.5322
Epoch 4/10
797/797 [==============================] - 12s 15ms/step - loss: 0.3436 - acc: 0.8758 - val_loss: 0.9026 - val_acc: 0.7135
Epoch 5/10
797/797 [==============================] - 12s 15ms/step - loss: 0.2924 - acc: 0.9021 - val_loss: 0.2977 - val_acc: 0.9181
Epoch 6/10
797/797 [==============================] - 12s 15ms/step - loss: 0.2673 - acc: 0.9197 - val_loss: 0.3671 - val_acc: 0.9123
Epoch 7/10
797/797 [==============================] - 12s 15ms/step - loss: 0.2155 - acc: 0.9210 - val_loss: 0.2847 - val_acc: 0.9240
Epoch 8/10
797/7

In [10]:
score = model.evaluate(x_test, y_test, verbose=1)

171/171 [==============================] - 1s 4ms/step


In [11]:
i = 0
for metric in model.metrics_names:
    print(metric + ": " + str(score[i]))
    i += 1

loss: 0.5187677096205148
acc: 0.877192983850401


In [13]:
client.upload_file('./human_not_human.h5', bucket, 'models/human_not_human.h5')